In [477]:
import requests
import bs4
from bs4 import BeautifulSoup as bs
import urllib.request as ureq
import csv
import re
import numpy as np
import pandas as pd
import nltk


In [404]:
Australiaforum_url = 'https://www.australiaforum.com/visas-immigration/'
Ausforum = requests.get(Australiaforum_url)
#print(Ausforum)
soup = bs(Ausforum.text, 'html.parser')
#print(soup.prettify())
#soup.find_all('a')
#tabs = [link['href'] for link in soup.find_all('a', {'class':"n_first_a"})
#print(tabs)
#tab_titles = list(map(lambda h: h.text.strip(), tabs))
#print(tab_titles)
#for links in tabs:
    #print links.get("href")
#THIS PRINTS ALL HREF LINKS
#for link in soup.find_all('a'):
    #print(link.get('href'))

#THIS PRINTS THE TOP TABS WITH CLASS: N_FIRST_A:
#for link in soup.find_all("a", "n_first_a", href=True):
 #   print(link['href'])
    


In [405]:
title = soup.find("a", {'title': (re.compile('Last Page'))}).get('href')

In [406]:
#last page
last_num = int(re.search(r"index(\d+)\.", title).groups()[0])
print(last_num)

1861


In [407]:


thread_page_urls = []

for i in range(0, last_num):
    thread_page_urls.append('https://www.australiaforum.com/visas-immigration/index'+ str(i) + '.html')
    

print(len(thread_page_urls))

1861


In [433]:
#thread titles from all pages

title_urls = []
counter = 0 

# For running the full data set
# max_urls = len(thread_page_urls)
max_urls = 693  # For production
max_urls = 2 
#0 and 1 are the same page
for i in range(1, max_urls):
    url = thread_page_urls[i]
    # Prints the URL every 10 iterations
    if counter % 10 == 0:
        print(str(counter) + ": " +url)
    thread_content = requests.get(url)
    thread_soup = bs(thread_content.text, 'html.parser')

    for link in thread_soup.find_all("a", {'id': (re.compile('thread_title_'))}):
        href = link.get('href')
        title_urls.append(href)              
    counter = counter + 1


0: https://www.australiaforum.com/visas-immigration/index1.html


In [434]:
thread_url = title_urls[0]
thread_page_count = get_thread_page_count(title_urls[0])
print(thread_url)

https://www.australiaforum.com/visas-immigration/250194-permanent-partner-visas-801-100-eligibility-notification.html


In [410]:
def get_thread_page_count(thread_url):
    page = requests.get(thread_url)
    soup = bs(page.content, 'html.parser')

    lastPage = 1
    pageNav = soup.select('div.pagenav')
    if len(pageNav) > 0:
        for s in pageNav[0].stripped_strings:
            m = re.search('^Page \d+ of (\d+)', s)
            if (m):
                groups = m.groups()
                lastPage = groups[0]
    return(int(lastPage))

In [435]:
def get_thread_messages(thread_url, total_posts, printing = False):
    groups = re.search(r"(.*)(\.html)", thread_url).groups()
    base_url = groups[0]
    suffix = groups[1]
    
    ids = []
    links = []
    dates = []
    usernames = []
    messages = []
    likes = []
    countries = []
    
    for i in range(1, total_posts+1):
        index = ''
        if i > 1:
            index = '-' + str(i)
        new_url = base_url + index + suffix
        if (printing == True):
            print(new_url)
        page = requests.get(new_url)
        soup = bs(page.content, 'html.parser')
        posts = soup.select('#posts table[id^="post"]')
        for i in posts:
            #print(i['id'])
            ids.append(i['id'])
            link = i.select('a[title="Link to this Post"]')
            #print(link[0]['href'])
            links.append(link[0]['href'])
            tds = i.find_all('td')
            #print(tds)
            dateStrings = tds[0].stripped_strings
            date = ''
            for j in dateStrings:
                m = re.search('^\d{2}-', j)
                if (m):
                    date = j
                    #print(j)
            dates.append(date)
            country = ''
            countryS = i.select('div.smallfont')
            for r in countryS:
                r = r.stripped_strings
                for c in r:
                    d = re.search('^From\ (.*)', c)
                    if (d):
                        country = d.group(1)
                        #print(country)
            countries.append(country)
            like = 0
            likeS = i.select('div.smallfont')
            for l in likeS:
                l = l.stripped_strings
                for u in l:
                    e = re.search('^(\d*)(\ likes received)', u)
                    if (e):
                        like = int(e.group(1))
            likes.append(like)
            usernameA = i.select('a.bigusername')
            #print(usernameA[0].string)
            usernames.append(usernameA[0].string)
            postId = '#post_message_' + i['id'][4:]
            postMsg = i.select(postId)
            concat_message = ''
            for s in postMsg[0].strings:
                # print(s)
                concat_message += s + ' ' 
            messages.append(concat_message)
    data = { "ID": ids, "Link": links, "Date": dates, "Username": usernames, "Message": messages, "Country": countries, "Like": likes}
    return(pd.DataFrame(data))

    #frame = pd.DataFrame.from_dict(data, orient='index')
    #frame = frame.transpose()
    #frame.to_csv('ausforum.csv')

In [437]:
# Create empty frame
base_data_frame = pd.DataFrame({ "ID": [], "Link": [], "Date": [], "Username": [], "Message": [], "Country": [], "Like": []})

# For all the threads, get all the messages
for thread_url in title_urls:
    print(thread_url)
    thread_page_count = get_thread_page_count(thread_url)
     #For testing, reset thread_page_count to a low value
    # thread_page_count = 3
    data_frame = get_thread_messages(thread_url, thread_page_count, printing=True)
    base_data_frame = base_data_frame.append(data_frame, ignore_index=True)
print('Finished')

https://www.australiaforum.com/visas-immigration/250194-permanent-partner-visas-801-100-eligibility-notification.html
https://www.australiaforum.com/visas-immigration/250194-permanent-partner-visas-801-100-eligibility-notification.html
https://www.australiaforum.com/visas-immigration/250194-permanent-partner-visas-801-100-eligibility-notification-2.html
https://www.australiaforum.com/visas-immigration/250194-permanent-partner-visas-801-100-eligibility-notification-3.html
https://www.australiaforum.com/visas-immigration/252474-registered-migration-agents.html
https://www.australiaforum.com/visas-immigration/252474-registered-migration-agents.html
https://www.australiaforum.com/visas-immigration/283163-help.html
https://www.australiaforum.com/visas-immigration/283163-help.html
https://www.australiaforum.com/visas-immigration/283163-help-2.html
https://www.australiaforum.com/visas-immigration/283163-help-3.html
https://www.australiaforum.com/visas-immigration/283163-help-4.html
https://ww

https://www.australiaforum.com/visas-immigration/280753-partner-visa-309-processing-time-beirut-office-58.html
https://www.australiaforum.com/visas-immigration/280753-partner-visa-309-processing-time-beirut-office-59.html
https://www.australiaforum.com/visas-immigration/280753-partner-visa-309-processing-time-beirut-office-60.html
https://www.australiaforum.com/visas-immigration/280753-partner-visa-309-processing-time-beirut-office-61.html
https://www.australiaforum.com/visas-immigration/38930-820-temporary-onshore-waiting-room.html
https://www.australiaforum.com/visas-immigration/38930-820-temporary-onshore-waiting-room.html
https://www.australiaforum.com/visas-immigration/38930-820-temporary-onshore-waiting-room-2.html
https://www.australiaforum.com/visas-immigration/38930-820-temporary-onshore-waiting-room-3.html
https://www.australiaforum.com/visas-immigration/38930-820-temporary-onshore-waiting-room-4.html
https://www.australiaforum.com/visas-immigration/38930-820-temporary-onshor

https://www.australiaforum.com/visas-immigration/38930-820-temporary-onshore-waiting-room-80.html
https://www.australiaforum.com/visas-immigration/38930-820-temporary-onshore-waiting-room-81.html
https://www.australiaforum.com/visas-immigration/38930-820-temporary-onshore-waiting-room-82.html
https://www.australiaforum.com/visas-immigration/38930-820-temporary-onshore-waiting-room-83.html
https://www.australiaforum.com/visas-immigration/38930-820-temporary-onshore-waiting-room-84.html
https://www.australiaforum.com/visas-immigration/38930-820-temporary-onshore-waiting-room-85.html
https://www.australiaforum.com/visas-immigration/38930-820-temporary-onshore-waiting-room-86.html
https://www.australiaforum.com/visas-immigration/38930-820-temporary-onshore-waiting-room-87.html
https://www.australiaforum.com/visas-immigration/38930-820-temporary-onshore-waiting-room-88.html
https://www.australiaforum.com/visas-immigration/38930-820-temporary-onshore-waiting-room-89.html
https://www.australi

https://www.australiaforum.com/visas-immigration/38930-820-temporary-onshore-waiting-room-163.html
https://www.australiaforum.com/visas-immigration/38930-820-temporary-onshore-waiting-room-164.html
https://www.australiaforum.com/visas-immigration/38930-820-temporary-onshore-waiting-room-165.html
https://www.australiaforum.com/visas-immigration/38930-820-temporary-onshore-waiting-room-166.html
https://www.australiaforum.com/visas-immigration/38930-820-temporary-onshore-waiting-room-167.html
https://www.australiaforum.com/visas-immigration/38930-820-temporary-onshore-waiting-room-168.html
https://www.australiaforum.com/visas-immigration/38930-820-temporary-onshore-waiting-room-169.html
https://www.australiaforum.com/visas-immigration/38930-820-temporary-onshore-waiting-room-170.html
https://www.australiaforum.com/visas-immigration/38930-820-temporary-onshore-waiting-room-171.html
https://www.australiaforum.com/visas-immigration/38930-820-temporary-onshore-waiting-room-172.html
https://ww

https://www.australiaforum.com/visas-immigration/38930-820-temporary-onshore-waiting-room-246.html
https://www.australiaforum.com/visas-immigration/38930-820-temporary-onshore-waiting-room-247.html
https://www.australiaforum.com/visas-immigration/38930-820-temporary-onshore-waiting-room-248.html
https://www.australiaforum.com/visas-immigration/38930-820-temporary-onshore-waiting-room-249.html
https://www.australiaforum.com/visas-immigration/38930-820-temporary-onshore-waiting-room-250.html
https://www.australiaforum.com/visas-immigration/38930-820-temporary-onshore-waiting-room-251.html
https://www.australiaforum.com/visas-immigration/38930-820-temporary-onshore-waiting-room-252.html
https://www.australiaforum.com/visas-immigration/38930-820-temporary-onshore-waiting-room-253.html
https://www.australiaforum.com/visas-immigration/38930-820-temporary-onshore-waiting-room-254.html
https://www.australiaforum.com/visas-immigration/38930-820-temporary-onshore-waiting-room-255.html
https://ww

...

https://www.australiaforum.com/visas-immigration/195794-partner-visa-applications-pretoria-south-africa-2.html
https://www.australiaforum.com/visas-immigration/195794-partner-visa-applications-pretoria-south-africa-3.html
https://www.australiaforum.com/visas-immigration/195794-partner-visa-applications-pretoria-south-africa-4.html
https://www.australiaforum.com/visas-immigration/195794-partner-visa-applications-pretoria-south-africa-5.html
https://www.australiaforum.com/visas-immigration/195794-partner-visa-applications-pretoria-south-africa-6.html
https://www.australiaforum.com/visas-immigration/195794-partner-visa-applications-pretoria-south-africa-7.html
https://www.australiaforum.com/visas-immigration/195794-partner-visa-applications-pretoria-south-africa-8.html
https://www.australiaforum.com/visas-immigration/195794-partner-visa-applications-pretoria-south-africa-9.html
https://www.australiaforum.com/visas-immigration/195794-partner-visa-applications-pretoria-south-africa-10.html


https://www.australiaforum.com/visas-immigration/229034-500-subsequent-entrant-dependent-visa-processing-time.html
https://www.australiaforum.com/visas-immigration/229034-500-subsequent-entrant-dependent-visa-processing-time-2.html
https://www.australiaforum.com/visas-immigration/229034-500-subsequent-entrant-dependent-visa-processing-time-3.html
https://www.australiaforum.com/visas-immigration/229034-500-subsequent-entrant-dependent-visa-processing-time-4.html
https://www.australiaforum.com/visas-immigration/229034-500-subsequent-entrant-dependent-visa-processing-time-5.html
https://www.australiaforum.com/visas-immigration/229034-500-subsequent-entrant-dependent-visa-processing-time-6.html
https://www.australiaforum.com/visas-immigration/229034-500-subsequent-entrant-dependent-visa-processing-time-7.html
https://www.australiaforum.com/visas-immigration/229034-500-subsequent-entrant-dependent-visa-processing-time-8.html
https://www.australiaforum.com/visas-immigration/229034-500-subseq

https://www.australiaforum.com/visas-immigration/229034-500-subsequent-entrant-dependent-visa-processing-time-71.html
https://www.australiaforum.com/visas-immigration/229034-500-subsequent-entrant-dependent-visa-processing-time-72.html
https://www.australiaforum.com/visas-immigration/229034-500-subsequent-entrant-dependent-visa-processing-time-73.html
https://www.australiaforum.com/visas-immigration/229034-500-subsequent-entrant-dependent-visa-processing-time-74.html
https://www.australiaforum.com/visas-immigration/229034-500-subsequent-entrant-dependent-visa-processing-time-75.html
https://www.australiaforum.com/visas-immigration/229034-500-subsequent-entrant-dependent-visa-processing-time-76.html
https://www.australiaforum.com/visas-immigration/229034-500-subsequent-entrant-dependent-visa-processing-time-77.html
https://www.australiaforum.com/visas-immigration/229034-500-subsequent-entrant-dependent-visa-processing-time-78.html
https://www.australiaforum.com/visas-immigration/229034-

https://www.australiaforum.com/visas-immigration/186578-485-subsequent-entrant-processing-time-51.html
https://www.australiaforum.com/visas-immigration/186578-485-subsequent-entrant-processing-time-52.html
https://www.australiaforum.com/visas-immigration/186578-485-subsequent-entrant-processing-time-53.html
https://www.australiaforum.com/visas-immigration/186578-485-subsequent-entrant-processing-time-54.html
https://www.australiaforum.com/visas-immigration/186578-485-subsequent-entrant-processing-time-55.html
https://www.australiaforum.com/visas-immigration/186578-485-subsequent-entrant-processing-time-56.html
https://www.australiaforum.com/visas-immigration/186578-485-subsequent-entrant-processing-time-57.html
https://www.australiaforum.com/visas-immigration/186578-485-subsequent-entrant-processing-time-58.html
https://www.australiaforum.com/visas-immigration/186578-485-subsequent-entrant-processing-time-59.html
https://www.australiaforum.com/visas-immigration/186578-485-subsequent-en

ConnectionError: HTTPSConnectionPool(host='www.australiaforum.com', port=443): Max retries exceeded with url: /visas-immigration/186578-485-subsequent-entrant-processing-time-122.html (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x1ad28cf240>: Failed to establish a new connection: [Errno 50] Network is down',))

In [438]:
len(base_data_frame)

11207

In [474]:
base_data_frame.columns.values

array(['ID', 'Link', 'Date', 'Username', 'Message', 'Country', 'Like'],
      dtype=object)

In [475]:
# from https://github.com/pandas-dev/pandas/issues/22610
import csv
with open("aus-forum.csv", "w", errors='surrogatepass') as _file:
    writer = csv.writer(_file)
    #writer.writerow(row)
    writer.writerow(base_data_frame.columns.values)
    for index, row in base_data_frame.iterrows():
        writer.writerow(row)

In [463]:
base_data_frame.iloc[445:446].Message.iloc[0]

'\n \n Quote: \n \n \n \n \r\n\t\t\t\t\tOriginally Posted by  Salbasio \n \n \n Just received my golden mail now \n \nDate submitted : 06/09/2018 \nReceived a fon call (interview for both sponsor and applicant) : 19/12/2018 \nDate of grant : 10/05/2019 \n \n \n \n Congratulations Salbasio! &#x1f60a;\ud83e\udd73\r\n\t\t '

In [476]:
base_data_frame.iloc[0:100]

,ID,Link,Date,Username,Message,Country,Like
0,post1722650,https://www.australiaforum.com/visas-immigrati...,"11-07-2017, 01:54 AM",CCMS,\r\n\t\t\t\r\n\t\t\tPlease note that the Depar...,australia,2311.0
1,post1740594,https://www.australiaforum.com/visas-immigrati...,"11-24-2017, 04:20 AM",Alice88,\r\n\t\t\t\r\n\t\t\tI didn't know so I lodged ...,italy,3.0
2,post1742377,https://www.australiaforum.com/visas-immigrati...,"11-26-2017, 02:18 AM",HRose313,\n \n Quote: \n \n \n \n \r\n\t\t\t\t\tOrigina...,usa,93.0
3,post1762561,https://www.australiaforum.com/visas-immigrati...,"12-20-2017, 03:25 PM",emilieisabella92,\r\n\t\t\t\r\n\t\t\tHi guys! \r\nI am new to ...,sweden,22.0
4,post1777498,https://www.australiaforum.com/visas-immigrati...,"01-09-2018, 08:28 AM",cassieandsam,\r\n\t\t\t\r\n\t\t\tWow!! Fantastic news! Cong...,australia,1.0
5,post1779450,https://www.australiaforum.com/visas-immigrati...,"01-11-2018, 09:54 AM",emilieisabella92,\n \n Quote: \n \n \n \n \r\n\t\t\t\t\tOrigina...,sweden,22.0
6,post1785890,https://www.australiaforum.com/visas-immigrati...,"01-19-2018, 05:35 PM",Ranc,\r\n\t\t\t\r\n\t\t\tThank you for offering you...,netherlands,1.0
7,post1800002,https://www.australiaforum.com/visas-immigrati...,"02-04-2018, 12:24 AM",LisakJohnson,\r\n\t\t\t\r\n\t\t\tThat's excellent news. Con...,uk,1.0
8,post1803834,https://www.australiaforum.com/visas-immigrati...,"02-07-2018, 08:30 AM",Relliott,\r\n\t\t\t\r\n\t\t\tHi- \r\nI've just put thr...,uk,1.0
9,post1805114,https://www.australiaforum.com/visas-immigrati...,"02-08-2018, 09:57 AM",Sheldonpk,\n \n Quote: \n \n \n \n \r\n\t\t\t\t\tOrigina...,australia,22.0
